# import the packages

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import seaborn as sns
import cv2
import keras
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Defining a function for plotting the count of data against each class in each directory

In [ ]:
def plotData(dirPath):
    
    cats_cases_dir = dirPath + '/cats/'
    dogs_cases_dir = dirPath + '/dogs/'

    # Get the list of all the images
    cats_cases_dir = glob.glob(cats_cases_dir + '*.jpg')
    dogs_cases_dir = glob.glob(dogs_cases_dir + '*.jpg')

    # An empty list. We will insert the data into this list in (img_path, label) format
    data1 = []

    # Go through all the cats images. The label for these cases will be 0
    for img in cats_cases_dir:
        data1.append((img, 'cats'))

    # Go through all the dogs images. The label for these cases will be 1
    for img in dogs_cases_dir:
        data1.append((img, 'dogs'))

    # Get a pandas dataframe from the data we have in our list
    data1 = pd.DataFrame(data1, columns=['image', 'label'], index=None)

    # Shuffle the data
    data1 = data1.sample(frac=1.).reset_index(drop=True)

    # Get the counts for each class
    cases_count = data1['label'].value_counts()
    print(cases_count)

    # Plot the results
    plt.figure(figsize=(10, 8))
    sns.barplot(x=cases_count.index, y=cases_count.values)
    plt.title('Number of cases', fontsize=14)
    plt.xlabel('Case type', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(range(len(cases_count.index)), ['cats(cats)', 'dogs(dogs)'])
    plt.show()

# usage of plotting function

In [ ]:
plotData('data/train')
plotData('data/test')
plotData('data/valid')

# Get few samples for both the classes

In [ ]:
cats_cases_dir = 'data/train/cats/'
dogs_cases_dir = 'data/train/dogs/'

# Get the list of all the images
cats_cases_dir = glob.glob(cats_cases_dir + '*.jpg')
dogs_cases_dir = glob.glob(dogs_cases_dir + '*.jpg')

# An empty list. We will insert the data into this list in (img_path, label) format
train_data1 = []

# Go through all the cats images. The label for these cases will be 0
for img in cats_cases_dir:
    train_data1.append((img,0))

# Go through all the dogs images. The label for these cases will be 1
for img in dogs_cases_dir:
    train_data1.append((img, 1))

# Get a pandas dataframe from the data we have in our list 
train_data1 = pd.DataFrame(train_data1, columns=['image', 'label'],index=None)

In [ ]:
dogs_samples = (train_data1[train_data1['label']==1]['image'].iloc[:5]).tolist()
cats_samples = (train_data1[train_data1['label']==0]['image'].iloc[:5]).tolist()

# Concat the data in a single list and del the above two list
samples = dogs_samples + cats_samples
del dogs_samples, cats_samples

# Plot the data 
f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = cv2.imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("cats")
    else:
        ax[i//5, i%5].set_title("dogs")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

# Defining a method to get the number of files given a path

In [ ]:
def retrieveNumberOfFiles(path): 
    cats_cases_dir = os.path.join(path, 'cats')
    dogs_cases_dir = os.path.join(path, 'dogs')
    list0 = os.listdir(cats_cases_dir) 
    list1 = os.listdir(dogs_cases_dir)  
    return len(list0) , len(list1)

# example of function usage

In [ ]:
totalTrain = retrieveNumberOfFiles('data/train')[0] + retrieveNumberOfFiles('data/train')[1]
totalVal = retrieveNumberOfFiles('data/valid')[0] + retrieveNumberOfFiles('data/valid')[1]
totalTest = retrieveNumberOfFiles('data/test')[0] + retrieveNumberOfFiles('data/test')[1]
print(totalTrain)

# Initialize the training and valid data augmentation object

In [ ]:
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

# Initialize the training generator
trainGen = trainAug.flow_from_directory(
	'data/train',
	class_mode="categorical",
	target_size=(244, 244),
	color_mode="rgb",
	shuffle=True,
	batch_size=16)

# for debugging usage
for images, labels in trainGen: 
    print(f"Images shape: {images.shape}, Labels shape: {labels.shape}") 
    print(f"Images dtype: {images.dtype}, Labels dtype: {labels.dtype}")
    break
images, labels = next(trainGen)
print("Images shape:", images.shape)  # (16, 244, 244, 3)
print("Labels shape:", labels.shape)  # (16, num_classes)

In [ ]:
valAug = ImageDataGenerator(rescale=1/255.0)

# Initialize the validation generator
valGen = valAug.flow_from_directory(
	'data/valid',
	class_mode="categorical",
	target_size=(244, 244),
	color_mode="rgb",
	shuffle=False,
	batch_size=16)

In [ ]:
# Initialize the testing generator
testGen = valAug.flow_from_directory(
	'data/test',
	class_mode="categorical",
	target_size=(244, 244),
	color_mode="rgb",
	shuffle=False,
	batch_size=16)

# building the custom CNN model

In [ ]:
# Importing packages

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# be sure about last layer output number should match num of class!
class BC_Model:
    @staticmethod
    def build(width, height, depth, classes):
    
        # Lets first initialize the model with input shape to be "channels last" and channel's dimension
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        # If we are using "channels first", then let's update the input shape and channel's dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # (CONV2D => RELU => BN ) * 1 => POOL => DROPOUT
        model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV2D => RELU => BN ) * 2 => POOL => DROPOUT
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV2D => RELU => BN ) * 3 => POOL => DROPOUT
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV2D => RELU => BN ) * 4 => POOL => DROPOUT
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # FC => RELU layers => BN => DROPOUT
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # Dense layer and softmax 'sigmoid' classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        # Returning the created network architecture
        return model


# create object from the CNN model class

In [ ]:
dogs_cats_cnn_model = BC_Model.build(width=244, height=244, depth=3, classes=2)
# input shap for debugging
print(dogs_cats_cnn_model.input_shape)
print(dogs_cats_cnn_model.output_shape)

# compiling the model and setting learning rate with optimizer 'Adam'

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)
opt = Adam(learning_rate=lr_schedule)
# integrate optimizer to CNN model
dogs_cats_cnn_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# setting hyperparameters for fitting the model (class_weight, check points,..)

In [ ]:
# Calculate class weights

train_counts = {'cats': retrieveNumberOfFiles('data/train')[0], 'dogs': retrieveNumberOfFiles('data/train')[1]}

# Labels for the training set
y_train = [0] * train_counts['cats'] + [1] * train_counts['dogs']  
# 0 for cats, 1 for dogs

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

# Convert to dictionary format
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

In [ ]:

checkpoint = ModelCheckpoint('best_model.keras', 
                             save_best_only=True, 
                             monitor='val_loss', 
                             mode='min', 
                             verbose=1)


early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=10,  
                               mode='min', 
                               verbose=1)

# fitting the model

In [ ]:
steps_per_epoch = trainGen.samples // trainGen.batch_size
validation_steps = valGen.samples // valGen.batch_size

# Start training
history = dogs_cats_cnn_model.fit(
    trainGen,
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # specify the number of epochs you want
    validation_data=valGen,
    validation_steps=validation_steps,
    callbacks=[checkpoint, early_stopping]  # include both callbacks
)

In [ ]:
# saving last epoch metrics
dogs_cats_cnn_model.save('final_model.keras')

# plotting the model training process

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# evaluate the Model using test set ('testGEN')

In [ ]:
# Evaluate model performance on the test set
test_loss, test_accuracy = dogs_cats_cnn_model.evaluate(
    testGen, 
    steps=testGen.samples // testGen.batch_size)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

# make predication using trained model

In [ ]:
# Load the best saved custom CNN model

best_model = load_model("best_model.keras")
image_path = "dog.jpg"
img = load_img(image_path, target_size=(244, 244))  # Use your input size
img_array = img_to_array(img)  # Convert to numpy array
img_array = img_array / 255.0  # Normalize if required
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Get predictions (probabilities for each class)
predictions = best_model.predict(img_array)

# Extract probability of the predicted class
predicted_class = np.argmax(predictions, axis=1)  # Class index
predicted_probability = predictions[0][predicted_class[0]]  # Probability of that class
print(testGen.class_indices)  # Dictionary mapping labels to indices
print(f"Predicted class: {predicted_class[0]}")
print(f"Predicted probability: {predicted_probability:.2f}")